### 2γ) Bayesian Networks με PGMPY
Δεν είναι αποτελεσματική καθώς δεν πιάνει υψηλό accuracy

In [22]:
import pgmpy
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=Warning)

pgmpy.__version__

'0.1.25'

In [23]:
CSV_PATH = os.path.join('harth')
df6 = pd.read_csv(CSV_PATH + "/S006.csv")
df20 = pd.read_csv(CSV_PATH + "/S020.csv")
df28 = pd.read_csv(CSV_PATH + "/S028.csv")
df15 = pd.read_csv(CSV_PATH + "/S015.csv")
ultimate_df = pd.concat([df6, df20, df28, df15])

data_bayes_train = ultimate_df[['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z', 'label']]
data_bayes_train.dropna()
data_bayes_train.drop_duplicates()

,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
0,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6
1,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6
2,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6
3,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6
4,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6
...,...,...,...,...,...,...,...
418387,-0.995198,0.003628,-0.013364,-0.948044,0.011695,-0.318949,6
418388,-0.989947,0.011355,-0.012790,-0.949454,0.013267,-0.322742,6
418389,-0.998318,-0.001806,-0.013925,-0.947970,0.004826,-0.318672,6
418390,-0.987699,0.010354,-0.011789,-0.949501,-0.006672,-0.323341,6


In [ ]:
from sklearn.model_selection import train_test_split

train_bayes, test_bayes = train_test_split(data_bayes_train, test_size=0.3, random_state=42)
train_bayes = train_bayes.sample(10000, random_state=42)
test_bayes = test_bayes.sample(10000, random_state=42)

Βοηθητικές Συναρτήσεις

In [25]:
from pgmpy.inference import VariableElimination
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_bn(df, bn):
    predictor = VariableElimination(bn)
    results = []
    classes = {}
    i = 0
    # Output Normalization
    for index, c in enumerate(df['label']):
        if c not in classes.values():
            classes[i] = c
            i+=1

    for index, row in df.iterrows():
        predicted_distribution = predictor.query(variables=['label'], evidence=None)
        predicted_label = predicted_distribution.values.argmax()
        results.append(classes[predicted_label])
    
    precision = precision_score(df['label'], results, average='weighted')
    recall = recall_score(df['label'], results, average='weighted')
    f1 = f1_score(df['label'], results, average='weighted')

    print(F"Precision: {precision:.4f}, Recall: {recall:.4f}, f1: {f1:.4f}")

    return precision, recall, f1
   

In [27]:
def ensure_consistent_states(data, state_names):
    for column in data.columns:
        if column in state_names:
            data[column] = pd.Categorical(data[column], categories=state_names[column])
        else:
            state_names[column] = data[column].unique().tolist()
    return data, state_names

Δημιουργούμε το Μοντέλο

In [ ]:
from pgmpy.models import BayesianNetwork
bayesian = BayesianNetwork([('back_x', 'label'), 
                            ('back_y', 'label'), 
                            ('back_z', 'label'), 
                            ('thigh_x', 'label'),
                            ('thigh_y', 'label'),
                            ('thigh_z', 'label'),
                            ('back_x', 'back_y'), 
                            ('back_y', 'back_z'), 
                            ('thigh_x', 'thigh_y'), 
                            ('thigh_y', 'thigh_z')
                            ])

Εκπαίδευση του Μοντέλου

In [29]:

state_names = {}
initial_data = train_bayes[:10]
subsequent_batches = [train_bayes[i:i+10] for i in range(10, len(train_bayes), 10)]
initial_data, state_names = ensure_consistent_states(initial_data, state_names)
bayesian.fit(initial_data)
n_prev_samples = len(initial_data)


for batch in subsequent_batches:
    batch, state_names = ensure_consistent_states(batch, state_names)
    try:
        bayesian.fit_update(batch, n_prev_samples=n_prev_samples)
        n_prev_samples += len(batch)
    except ValueError as e:
        print(f"Error updating with batch: {e}")


bayesian.check_model()


True

Αξιολόγηση

In [30]:
prec_m, recl_m, f1_m = evaluate_bn(test_bayes, bayesian)


Precision: 0.2510, Recall: 0.5010, f1: 0.3344
